# Basis functions for all points

Block size:

chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/http://www.saimm.org.za/Journal/v106n03p205.pdf

In [ ]:
import pandas as pd
from dk_model import DeepKrigingTrainer
import numpy as np
import gstools as gs

import scipy.stats as stats
##Library neural nets
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import keras

import matplotlib as mpl
mpl.style.use("seaborn")
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
get_ipython().magic(u"config InlineBackend.figure_format = 'svg'")
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
import pylab

deposit_data = pd.read_csv("Data/final_dataset_1.csv", low_memory=False)


In [ ]:
x_min, x_max = deposit_data['X'].min(), deposit_data['X'].max()   
y_min, y_max = deposit_data['Y'].min(), deposit_data['Y'].max()   
z_min, z_max = deposit_data['Z'].min(), deposit_data['Z'].max()   

x = np.linspace(x_min, x_max, 25)
y = np.linspace(y_min, y_max, 25)
z = np.linspace(z_min, z_max, 25)

X, Y, Z = np.meshgrid(x, y, z)

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T



In [ ]:
N = len(grid_points)

lon = grid_points[:, 0]
lat = grid_points[:, 1]
az = grid_points[:, 2]

num_basis_1_lvl = [10**3]

num_basis_list = [num_basis_1_lvl]

phi_arrays = []  

# For each grid
for grid in num_basis_list:
    knots_1dx = [np.linspace(0, 1, int(i**(1/3)) + 1) for i in grid]
    knots_1dy = [np.linspace(0, 1, int(i**(1/3)) + 1) for i in grid]
    knots_1dz = [np.linspace(0, 1, int(i**(1/3)) + 1) for i in grid]
    basis_size = 0
    phis = np.zeros((N, sum(grid)))
    
    # For each level of resolution
    for res in range(len(grid)):
        theta = 1 / (grid[res]**(1/3)) * 2.5
        knots_x, knots_y, knots_z = np.meshgrid(knots_1dx[res], knots_1dy[res], knots_1dz[res])
        knots = np.column_stack((knots_x.flatten(), knots_y.flatten(), knots_z.flatten()))
        
        # For each node in the grid
        for i in range(grid[res]):
            d = np.linalg.norm(np.vstack((lon, lat, az)).astype(float).T - knots[i, :], axis=1) / theta
            
            # For each distance of our data to the node i, calculate Wendland kernel
            for j in range(len(d)):
                if 0 <= d[j] <= 1:
                    phis[j, i + basis_size] = (1 - d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3) / 3
                else:
                    phis[j, i + basis_size] = 0
        
        basis_size += grid[res]
    
    phi_arrays.append(phis) 

phi_1_lvl = phi_arrays


In [ ]:
phi_1_lvl_df = pd.DataFrame(phi_1_lvl[0])

# Remove all-zero columns
#phi_1_lvl_df = phi_1_lvl_df.loc[:, (phi_1_lvl_df != 0).any(axis=0)]

new_columns = {i: f'phi_{i}' for i in range(phi_1_lvl_df.shape[1])}

phi_1_lvl_df.columns = pd.Index(new_columns.values())

xyz_df = pd.DataFrame(grid_points)
xyz_df.columns = ['X', 'Y', 'Z']


In [ ]:

y = np.array(deposit_data['Density_gcm3']).reshape(-1,1)
phi = np.array(deposit_data.iloc[:, 10:])
N = len(deposit_data)
indices = np.arange(N)
Xs = np.array(deposit_data.iloc[:, 1:4])

from sklearn.model_selection import train_test_split
indices = np.arange(N)
Xs_train, Xs_test, phi_train, phi_test, y_train, y_test, idx_train, idx_test     = train_test_split(Xs, phi, y, indices, test_size=0.2)
N_train = Xs_train.shape[0]
N_test = Xs_test.shape[0]

In [ ]:
Xs_pred = np.array(xyz_df)
phi_pred = np.array(phi_1_lvl_df)

In [ ]:
grid_points_df= pd.DataFrame(grid_points)
grid_points_df[['X','Y','Z']] = pd.DataFrame(grid_points)
grid_points_df = grid_points_df[['X','Y','Z']] 

filtered_unnormalised_deposit_data = pd.read_csv('Data/filtered_unnormalised_deposit_data.csv')
covariates=['X', 'Y', 'Z', 'CP_Total', 'PO_Total', 'PY_Total', 'RQD_Pct', 'Cr_ppm']
# Calculate the multiplier for each column
multipliers = {}
for column in covariates + ['Density_gcm3']:
    ratio = deposit_data[column] / filtered_unnormalised_deposit_data[column]
    multiplier = ratio.mean()
    multipliers[column] = 1/multiplier

multipliers_df = pd.DataFrame.from_dict(multipliers, orient='index', columns=['Multiplier'])

multipliers['X'] = filtered_unnormalised_deposit_data['X'].max() - filtered_unnormalised_deposit_data['X'].min()
multipliers['Y'] = filtered_unnormalised_deposit_data['Y'].max() - filtered_unnormalised_deposit_data['Y'].min()
multipliers['Z'] = filtered_unnormalised_deposit_data['Z'].max() - filtered_unnormalised_deposit_data['Z'].min()

In [ ]:
from keras.layers import Dense, Dropout,BatchNormalization
import tensorflow as tf
# seed = 42
# tf.random.set_seed(seed)
# np.random.seed(seed)


def create_mlp(feature_dim):
    # create model
    model = Sequential()
    model.add(Dense(200, input_dim = feature_dim,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='relu'))

    model.compile(loss='poisson', optimizer='adam', metrics=['mse','mae'])
    return model

In [ ]:
model_nn = create_mlp(feature_dim = 3)
print("[INFO] training model nn...")
train_history = model_nn.fit(Xs_train, y_train, validation_split = 0.2, epochs = 500, batch_size = 32, verbose = 0)

In [ ]:
model_dk = create_mlp(feature_dim = 1000)
print("[INFO] training model dk...")
train_history_dk = model_dk.fit(phi_train, y_train, validation_split = 0.2, epochs = 500, batch_size = 32, verbose = 0)

In [ ]:
nn_test_pred = model_nn.predict(Xs_test)
mse = np.mean((nn_test_pred - y_test) ** 2)

dk_test_pred = model_dk.predict(phi_test)
mse_dk = np.mean((dk_test_pred - y_test) ** 2)


print(f"MSE: {mse:.4f}, {mse_dk:.4f}")

In [ ]:
interp = model_dk.predict(phi_pred)

In [ ]:
interpnn = model_nn.predict(Xs_pred)

In [ ]:
# np.random.seed(42)


# # Use GSTools for Ordinary Kriging
# model = gs.Gaussian(dim=3)
# krige = gs.krige.Ordinary(model, cond_pos=[Xs_train[:, 0], Xs_train[:, 1], Xs_train[:, 2]], cond_val=y_train, fit_variogram=True)

# # Execute kriging at the points of xyz_df
# kriging_pred, kriging_var = krige([xyz_df['X'].values, xyz_df['Y'].values, xyz_df['Z'].values], return_var=True )

# # Display kriging predictions
# print(kriging_pred)


In [ ]:
def mc_dropout_predict(model, x, n_iter=100):
    preds = []
    for i in range(n_iter):
        y_p = model(x, training=True) # enable dropout
        preds.append(y_p.numpy())
    preds = np.array(preds)
    mean_preds = preds.mean(axis=0)
    std_preds = preds.std(axis=0)
    return mean_preds, std_preds

mean_preds, std_preds = mc_dropout_predict(model_dk, phi_pred, n_iter=100)
mean_preds_nn, std_preds_nn = mc_dropout_predict(model_nn, Xs_pred, n_iter=100)

print("Predicted means:", mean_preds.flatten())
print("Predicted uncertainties (std):", std_preds.flatten())

In [ ]:
combined_data = pd.read_csv('Data/3d_grid_interpolations_var.csv')

In [ ]:
krig_std = np.sqrt(combined_data['Kriging_Var'].values)
plt.hist(krig_std, bins=300, edgecolor='black')
plt.title('Histogram of std_preds')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.hist(std_preds*multipliers['Density_gcm3'], bins=300, edgecolor='black')
plt.title('Histogram of std_preds')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# test_data_xyz = grid_points_df[['X', 'Y', 'Z']].values

# deep_kriging_df = pd.DataFrame(
#     np.hstack([test_data_xyz, interp.ravel()[:, None]]),
#     columns=['X', 'Y', 'Z', 'DeepKriging_Pred']
# )

# nn_df = pd.DataFrame(
#     np.hstack([test_data_xyz, interpnn.ravel()[:, None]]),
#     columns=['X', 'Y', 'Z', 'NN_Pred']
# )

# kriging_df = pd.DataFrame(
#     np.hstack([test_data_xyz, kriging_pred.ravel()[:, None], kriging_var.ravel()[:, None]]),
#     columns=['X', 'Y', 'Z', 'Kriging_Pred', 'Kriging_Var']
# )

# combined_data = deep_kriging_df.merge(nn_df, on=['X', 'Y', 'Z'], how='inner') \
#                                .merge(kriging_df, on=['X', 'Y', 'Z'], how='inner')

# combined_data[['DeepKriging_Pred', 'NN_Pred', 'Kriging_Pred']] *= multipliers['Density_gcm3']

# combined_data['Kriging_Var'] *= multipliers['Density_gcm3'] ** 2

# # **Add standard deviations to combined_data**
# combined_data['DeepKriging_Std'] = std_preds * multipliers['Density_gcm3']
# combined_data['NN_Std'] = std_preds_nn * multipliers['Density_gcm3']

# file_path = "3d_grid_interpolations_var.csv"
# combined_data.to_csv(file_path, index=False)

# print(f"Data saved to: {file_path}")


In [ ]:
combined_data['DeepKriging_Std'] = std_preds * multipliers['Density_gcm3']
combined_data['NN_Std'] = std_preds_nn * multipliers['Density_gcm3']

combined_data['DeepKriging_Pred'] = mean_preds*multipliers['Density_gcm3']
combined_data['NN_Pred'] = mean_preds_nn*multipliers['Density_gcm3']

In [ ]:
# combined_data['DeepKriging_Pred'] = interp*multipliers['Density_gcm3']
# combined_data['NN_Pred'] = interpnn*multipliers['Density_gcm3']

In [ ]:
x_middle = combined_data['X'].median()
y_middle = combined_data['Y'].median()

xz_plane_data = combined_data[combined_data['Y'] == y_middle]

yz_plane_data = combined_data[combined_data['X'] == x_middle]

z_plane_data = combined_data[combined_data['Z'] == 0]   
x_plane_data = combined_data[combined_data['X'] == 0]   
y_plane_data = combined_data[combined_data['Y'] == 1.0]

plane_data_xzyz = pd.concat([xz_plane_data, yz_plane_data, z_plane_data, x_plane_data, y_plane_data])

x_middle = combined_data['X'].median()
z_middle = combined_data['Z'].median()

xy_plane_data = combined_data[combined_data['Z'] == z_middle]

yz_plane_data = combined_data[combined_data['X'] == x_middle]

z_plane_data = combined_data[combined_data['Z'] == 0]   
x_plane_data = combined_data[combined_data['X'] == 0]   
y_plane_data = combined_data[combined_data['Y'] == 1.0] 

plane_data_xyyz = pd.concat([xy_plane_data, yz_plane_data, z_plane_data, x_plane_data, y_plane_data])

y_middle = combined_data['Y'].median()
z_middle = combined_data['Z'].median()

xy_plane_data = combined_data[combined_data['Z'] == z_middle]

xz_plane_data = combined_data[combined_data['Y'] == y_middle]

z_plane_data = combined_data[combined_data['Z'] == 0]  
x_plane_data = combined_data[combined_data['X'] == 0]   
y_plane_data = combined_data[combined_data['Y'] == 1.0] 

plane_data_xyxz = pd.concat([xy_plane_data, xz_plane_data, z_plane_data, x_plane_data, y_plane_data])

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, Normalize
import matplotlib.pyplot as plt

def filter_outliers(data_xyz, data_values, threshold):
    mean = np.mean(data_values)
    std_dev = np.std(data_values)
    mask = np.abs(data_values - mean) <= threshold * std_dev
    filtered_xyz = data_xyz[mask]
    filtered_values = data_values[mask]
    return filtered_xyz, filtered_values

datasets = [plane_data_xzyz, plane_data_xyyz, plane_data_xyxz]

colors = [(1, 0, 1), (0, 1, 1)]  # Magenta to Cyan
custom_cmap = LinearSegmentedColormap.from_list('MagentaCyan', colors)

threshold = 50

all_values = []

filtered_data = []
for data in datasets:
    test_data_xyz = data[['X', 'Y', 'Z']].values * [
        multipliers['X'], multipliers['Y'], multipliers['Z']
    ]
    interp = data['DeepKriging_Pred'].values
    interpnn = data['NN_Pred'].values
    kriging_pred = data['Kriging_Pred'].values

    filtered_test_xyz_1, filtered_interp = filter_outliers(test_data_xyz, interp, threshold)
    filtered_test_xyz_2, filtered_interpnn = filter_outliers(test_data_xyz, interpnn, threshold)
    filtered_test_xyz_3, filtered_kriging = filter_outliers(test_data_xyz, kriging_pred, threshold)

    all_values.extend([filtered_interp, filtered_interpnn, filtered_kriging])

    filtered_data.append([(filtered_test_xyz_2, filtered_interpnn),
                          (filtered_test_xyz_3, filtered_kriging),
                          (filtered_test_xyz_1, filtered_interp)])

filtered_train_xyz, filtered_y = filter_outliers(
    Xs_train * [multipliers['X'], multipliers['Y'], multipliers['Z']],
    y_train.ravel(), threshold
)

vmin, vmax = np.min(all_values), np.max(all_values)

fig, axs = plt.subplots(3, 4, figsize=(24, 15), subplot_kw={'projection': '3d'}, 
                        gridspec_kw={'wspace': 0.4, 'hspace': 0}) 

for ax_row in axs:
    for ax in ax_row:
        ax.set_facecolor((0, 0, 0, 0))  


titles = ['Training data', 'Deep neural net', 'Ordinary kriging', 'Deepkriging']

rotations = [(30, -60), (30, 350), (30, 280)]  
for i, (elev, azim) in enumerate(rotations):
    axs[i, 0].scatter(
        filtered_train_xyz[:, 0], filtered_train_xyz[:, 1], filtered_train_xyz[:, 2],
        c=filtered_y * multipliers['Density_gcm3'], cmap=custom_cmap, s=8, vmin=vmin, vmax=vmax
    )
    axs[i, 0].view_init(elev=elev, azim=azim)  
    if i == 0:
        axs[i, 0].set_title('Training data', fontsize=12)
    axs[i, 0].set_xlabel('Easting [m]', fontsize=8)
    axs[i, 0].set_ylabel('Northing [m]', fontsize=8)
    axs[i, 0].set_zlabel('Altitude [m]', fontsize=8)
    axs[i, 0].tick_params(labelsize=7)
    axs[i, 0].grid(False)

for i, row_data in enumerate(filtered_data):
    for j, (xyz, values) in enumerate(row_data):
        axs[i, j + 1].scatter(
            xyz[:, 0], xyz[:, 1], xyz[:, 2],
            c=values, cmap=custom_cmap, s=28, vmin=vmin, vmax=vmax
        )
        if i == 0:  
            axs[i, j + 1].set_title(titles[j + 1], fontsize=12)
        axs[i, j + 1].set_xlabel('Easting [m]', fontsize=8)
        axs[i, j + 1].set_ylabel('Northing [m]', fontsize=8)
        axs[i, j + 1].set_zlabel('Altitude [m]', fontsize=8)
        axs[i, j + 1].tick_params(labelsize=7)
        axs[i, j + 1].grid(False)

norm = Normalize(vmin=vmin, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=axs, pad=0.05, aspect=50, extend='both', location='right')
cbar.set_label('Predicted density (g/cm³)', fontsize=10)
cbar.ax.tick_params(labelsize=8)

#plt.tight_layout()
plt.show()
fig.savefig("Figures/predictions_plot.png", dpi=300, bbox_inches='tight')



In [ ]:
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, Normalize
import matplotlib.pyplot as plt

def separate_outliers(data_xyz, data_values, threshold):
    mean = np.mean(data_values)
    std_dev = np.std(data_values)
    mask = np.abs(data_values - mean) <= threshold * std_dev
    inliers_xyz = data_xyz[mask]
    inliers_values = data_values[mask]
    outliers_xyz = data_xyz[~mask]
    outliers_values = data_values[~mask]
    return inliers_xyz, inliers_values, outliers_xyz, outliers_values

datasets = [plane_data_xzyz, plane_data_xyyz, plane_data_xyxz]

colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue -> White -> Red
custom_cmap = LinearSegmentedColormap.from_list('BlueWhiteRed', colors)


# Threshold for outlier filtering
threshold = 50

# Initialize variables to store global min and max values
all_values = []

filtered_data = []
for data in datasets:
    test_data_xyz = data[['X', 'Y', 'Z']].values * [
        multipliers['X'], multipliers['Y'], multipliers['Z']
    ]
    interp = data['DeepKriging_Std'].values
    interpnn = data['NN_Std'].values
    kriging_pred = np.sqrt(data['Kriging_Var'].values)

    inliers_test_xyz_nn, inliers_interpnn, outliers_test_xyz_nn, outliers_interpnn = separate_outliers(test_data_xyz, interpnn, threshold)

    # Keep the full range for DeepKriging and Ordinary Kriging data
    filtered_test_xyz_1, filtered_interp = test_data_xyz, interp
    filtered_test_xyz_3, filtered_kriging = test_data_xyz, kriging_pred

    all_values.extend([filtered_interp, inliers_interpnn, filtered_kriging])

    filtered_data.append([
        ((inliers_test_xyz_nn, inliers_interpnn), (outliers_test_xyz_nn, outliers_interpnn)),  # NN std (filtered + outliers)
        (filtered_test_xyz_3, filtered_kriging),   # Ordinary Kriging std (unfiltered)
        (filtered_test_xyz_1, filtered_interp)     # DeepKriging std (unfiltered)
    ])

filtered_train_xyz, filtered_y = separate_outliers(
    Xs_train * [multipliers['X'], multipliers['Y'], multipliers['Z']],
    y_train.ravel(), threshold
)[:2] 

# Compute global min and max values for color scaling
#vmin, vmax = np.min(all_values), np.max(all_values)
vmin, vmax = np.percentile(all_values, [0.2, 99.8])  # Using 0.2th and 99.8th percentiles to exclude outliers

# Create a 3x4 Matplotlib figure with tighter spacing
fig, axs = plt.subplots(3, 4, figsize=(24, 15), subplot_kw={'projection': '3d'},
                        gridspec_kw={'wspace': 0.4, 'hspace': 0})

for ax_row in axs:
    for ax in ax_row:
        ax.set_facecolor((0, 0, 0, 0)) 

titles = ['Training data', 'Deep neural net standard deviation', 'Ordinary kriging standard deviation', 'Deepkriging standard deviation']

# First column: Training Data with slight rotations
rotations = [(30, -60), (30, 350), (30, 280)]  
for i, (elev, azim) in enumerate(rotations):
    axs[i, 0].scatter(
        filtered_train_xyz[:, 0], filtered_train_xyz[:, 1], filtered_train_xyz[:, 2],
        color='green', s=8  
    )
    axs[i, 0].view_init(elev=elev, azim=azim)  # Apply rotation
    if i == 0:
        axs[i, 0].set_title('Training data', fontsize=12)
    axs[i, 0].set_xlabel('Easting [m]', fontsize=8)
    axs[i, 0].set_ylabel('Northing [m]', fontsize=8)
    axs[i, 0].set_zlabel('Altitude [m]', fontsize=8)
    axs[i, 0].tick_params(labelsize=7)
    axs[i, 0].grid(False)

for i, row_data in enumerate(filtered_data):
    for j, entry in enumerate(row_data):
        if j == 0:  
            (inliers_xyz, inliers_values), (outliers_xyz, outliers_values) = entry
            # Plot inliers with color map
            axs[i, j + 1].scatter(
                inliers_xyz[:, 0], inliers_xyz[:, 1], inliers_xyz[:, 2],
                c=inliers_values, cmap=custom_cmap, s=17, vmin=vmin, vmax=vmax
            )
            # Plot outliers in gray
            axs[i, j + 1].scatter(
                outliers_xyz[:, 0], outliers_xyz[:, 1], outliers_xyz[:, 2],
                color='gray', s=17
            )
        else:
            xyz, values = entry
            axs[i, j + 1].scatter(
                xyz[:, 0], xyz[:, 1], xyz[:, 2],
                c=values, cmap=custom_cmap, s=17, vmin=vmin, vmax=vmax
            )
        if i == 0:
            axs[i, j + 1].set_title(titles[j + 1], fontsize=12)
        axs[i, j + 1].set_xlabel('Easting [m]', fontsize=8)
        axs[i, j + 1].set_ylabel('Northing [m]', fontsize=8)
        axs[i, j + 1].set_zlabel('Altitude [m]', fontsize=8)
        axs[i, j + 1].tick_params(labelsize=7)
        axs[i, j + 1].grid(False)

# Create a shared colorbar with arrows 
norm = Normalize(vmin=vmin, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=axs, pad=0.05, aspect=50, extend='both', location='right')
cbar.set_label('Standard Deviation (g/cm³)', fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.tight_layout()
plt.show()
fig.savefig("Figures/uncertainty_plots10.png", dpi=300, bbox_inches='tight')
